## Tit

In [42]:
import util as Util
import classifiers
import preprocessing as pp
import sys
import csv
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score
from sklearn import cross_validation

from nltk.corpus import stopwords
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import sent_tokenize, RegexpTokenizer
import nltk
import distance

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn import svm
from sklearn.linear_model import perceptron
import xgboost as xgb

In [60]:
TRAIN_FILE='files/train.csv'
TEST_FILE='files/test.csv'
TRAIN_FEATURES='files/train_features.csv'
TEST_FEATURES='files/test_features.csv'

def logistic_regression_train(X, y):
	lr = LogisticRegression(C=1)	 
	#lr.fit(X, y)

	#Validacao do modelo
	scores = cross_val_score(lr, X, y, cv=10)
	print("Accuracy Logistic Regression: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
    
def xgboost_train(X, y):
    clf = xgb.XGBClassifier(learning_rate=0.15, n_estimators=170, nthread=6, max_depth=8, seed=0, silent=True,
                            subsample=0.85, colsample_bytree=0.85)
    
    #X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.1, random_state=1)
    
    #clf.fit(X_train, y_train)
    #y_pred = clf.predict_proba(X_test)
    
    scores = cross_val_score(clf, X, y, scoring='accuracy', cv=10)
    print("Accuracy XGBoost: %0.2f (+/- %0.2f)" % (scores.mean()*100, scores.std() * 2))    
    
    #print('XGBoost log loss: ', log_loss(y, y_pred))    
    #print("Score XGBoost network score: ",clf.score(X_test, y_test)*100)

def svm_train(X, y):
    #model4 = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)
    #print("Score SVM : ",model4.score(X_test, y_test)*100)
    
    scores = cross_val_score(svm.SVC(kernel='linear', C=1), X, y, scoring='accuracy', cv=10)
    print("Accuracy SVM: %0.2f (+/- %0.2f)" % (scores.mean()*100, scores.std() * 2))    

def perceptron_train(X, y):
    p = perceptron.Perceptron(n_iter=100, verbose=0, random_state=None, fit_intercept=True, eta0=0.002)
    #model5.fit(X_train,y_train)
    #print("Score Perceptron : ",model5.score(X_test, y_test)*100)
    
    scores = cross_val_score(perceptron.Perceptron(n_iter=100, verbose=0, random_state=None, fit_intercept=True, eta0=0.002), X, y, scoring='accuracy', cv=10)
    print("Accuracy Perceptron: %0.2f (+/- %0.2f)" % (scores.mean()*100, scores.std() * 2))  
    
    #return model5

In [61]:
def tokenize_cv(q):
	sentences = sent_tokenize(q)
	tokens = []
	
	for s in sentences:
		tokenizer = RegexpTokenizer(r'\w+')
		tokens += tokenizer.tokenize(s.lower())	

	st = PorterStemmer()
	return [st.stem(x) for x in tokens]

In [62]:
def import_data(n_examples):
	print("*** generating generate_cv ***")
	X_train = []
	y_train = []

	i = 0

	with open(TRAIN_FILE, "r") as f:
		csv_reader = csv.reader(f, delimiter=',', quotechar='"')
		next(csv_reader) # jumping the header stuff
		for index, line in enumerate(csv_reader):
			X_train.append(line[3] + " " + line[4])
			y_train.append(line[5])
			if(i % 1000 == 0): print(str(i) + '\r', end='')
			i += 1
			if(n_examples > 0 and i == n_examples): break

	return X_train, y_train

### Treinamento

In [64]:
from sklearn.feature_extraction.text import CountVectorizer

X_train, y = import_data(1000)

print("*** without stopwords and without stemming***")

vectorizer = CountVectorizer(binary=True,min_df=1,ngram_range=(1,1))
X = vectorizer.fit_transform(X_train, None)

logistic_regression_train(X, y)
#xgboost_train(X, y)
svm_train(X, y)
perceptron_train(X, y)

print("*** with stopwords and without stemming***")

vectorizer = CountVectorizer(binary=True,min_df=1,ngram_range=(1,1), stop_words='english')
X = vectorizer.fit_transform(X_train, None)

logistic_regression_train(X, y)
#xgboost_train(X, y)
svm_train(X, y)
perceptron_train(X, y)

print("*** without stopwords and with stemming***")

vectorizer = CountVectorizer(tokenizer=tokenize_cv, binary=True, min_df=1)
X = vectorizer.fit_transform(X_train, None)

logistic_regression_train(X, y)
#xgboost_train(X, y)
svm_train(X, y)
perceptron_train(X, y)

print("*** with stopwords and with stemming***")

vectorizer = CountVectorizer(tokenizer=tokenize_cv, binary=True, min_df=1, stop_words='english')
X = vectorizer.fit_transform(X_train, None)

logistic_regression_train(X, y)
#xgboost_train(X, y)
svm_train(X, y)
perceptron_train(X, y)

*** generating generate_cv ***
*** without stopwords and without stemming***
Accuracy Logistic Regression: 0.64 (+/- 0.10)
Accuracy SVM: 59.70 (+/- 0.08)
Accuracy Perceptron: 61.30 (+/- 0.10)
*** with stopwords and without stemming***
Accuracy Logistic Regression: 0.63 (+/- 0.08)
Accuracy SVM: 57.40 (+/- 0.06)
Accuracy Perceptron: 59.60 (+/- 0.09)
*** without stopwords and with stemming***
Accuracy Logistic Regression: 0.64 (+/- 0.05)
Accuracy SVM: 60.00 (+/- 0.09)
Accuracy Perceptron: 60.50 (+/- 0.07)
*** with stopwords and with stemming***
Accuracy Logistic Regression: 0.63 (+/- 0.06)
Accuracy SVM: 59.30 (+/- 0.10)
Accuracy Perceptron: 61.20 (+/- 0.09)
